# Import

In [1]:
import os

#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
print(tf.__version__)


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

2.7.0


In [2]:
# The CIFAR-10 dataset (Canadian Institute For Advanced Research) is a collection of images 
# that are commonly used to train machine learning and computer vision algorithms

# generate original training and test data
# The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, 
# with 6000 images per class. 
# There are 50000 training images and 10000 test images.
img_size = 32
img_chan = 3
n_classes = 10

learning_rate = 0.05
#global_step = 
input_size = 32*32*3
output_size = 10


print('\nLoading CIFAR10')

cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#x_train = np.reshape(x_train, [-1, img_size, img_size, img_chan])
x_train = np.reshape(x_train, [-1, img_size*img_size*3])
x_train = x_train.astype(np.float32) / 255
#x_test = np.reshape(X_test, [-1, img_size, img_size, img_chan])
x_test = np.reshape(x_test, [-1, img_size*img_size*3])
x_test = x_test.astype(np.float32) / 255

to_categorical = tf.keras.utils.to_categorical 
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print('\nSpliting data')

ind = np.random.permutation(x_train.shape[0])
x_train, y_train = x_train[ind], y_train[ind] 

VALIDATION_SPLIT = 0.1
n = int(x_train.shape[0] * (1-VALIDATION_SPLIT))
x_valid = x_train[n:]
x_train = x_train[:n]
y_valid = y_train[n:]
y_train = y_train[:n] 


train_num_examples = x_train.shape[0]
valid_num_examples = x_valid.shape[0]
test_num_examples  = x_test.shape[0]

print(train_num_examples, valid_num_examples, test_num_examples)


Loading CIFAR10

Spliting data
45000 5000 10000


In [3]:
#CIFAR data image of shape 32*32*3=3072
#CIFAR_size = 32*32*3
X = tf.placeholder(tf.float32, shape=[None, 3072])
Z = tf.placeholder(tf.float32, shape=[None, 100])

# Set up parameters for generator and discriminator

In [4]:
# Generator
# Define the variables for the generator, we will use them to build layers later
# -------------------
size_g_w1 = 100
size_g_b1 = 512
# A good way to decide the std for initializing the weights
w1_std = 1.0/tf.sqrt(size_g_w1/2.0)

G_W1 = tf.Variable(tf.random_normal(shape=[size_g_w1, size_g_b1], stddev=w1_std))
G_b1 = tf.Variable(tf.zeros(shape=[size_g_b1]))

size_g_w2 = 512
size_g_b2 = 3072
w2_std = 1.0/tf.sqrt(size_g_w2/2.0)

G_W2 = tf.Variable(tf.random_normal(shape=[size_g_w2, size_g_b2], stddev=w2_std))
G_b2 = tf.Variable(tf.zeros(shape=[size_g_b2]))
# theta_G and theta_D will be feeded to different optimizers later as "var_list", 
# since currently we have two networks instead of one now.
theta_G = [G_W1, G_W2, G_b1, G_b2]

# ====================
# Discriminator
# Define the variables for the discriminator
# --------------------
size_d_w1 = 3072
size_d_b1 = 512
w1_std = 1.0/tf.sqrt(size_d_w1/2.0)

D_W1 = tf.Variable(tf.random_normal(shape=[size_d_w1,size_d_b1], stddev=w1_std))
D_b1 = tf.Variable(tf.zeros(shape=[size_d_b1]))

size_d_w2 = 512
size_d_b2 = 1
w2_std = 1.0/tf.sqrt(size_d_w2/2.0)

D_W2 = tf.Variable(tf.random_normal(shape=[size_d_w2,size_d_b2], stddev=w2_std))
D_b2 = tf.Variable(tf.zeros(shape=[size_d_b2]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

# Generator module

In [5]:
def generator(z):

    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_logit = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_logit)

    return G_prob, G_logit


# Discriminator module

In [6]:
def discriminator(x):

    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

# Generate samples function

In [7]:
def sample_z(m, n):
    # randomly generate samples for generator
    return np.random.uniform(-1.0, 1.0, size = [m, n])

# Plotting samples

In [8]:
def plot_sample(samples, size1, size2):
    
    fig1 = plt.figure(figsize=(size1, size2))
    gs = gridspec.GridSpec(size1, size2)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        sample_reshaped=sample.reshape(32,32,3)
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample_reshaped[:,:,0], cmap='gray')
        plt.imshow(sample_reshaped[:,:,1], cmap='gray')
        plt.imshow(sample_reshaped[:,:,2], cmap='gray')

    return fig1

# Faciliate the path defining process

In [9]:
# Though it's not possible to get the path to the notebook by __file__, os.path is still very useful in dealing with paths and files
# In this case, we can use an alternative: pathlib.Path
"""
code_dir   = os.path.dirname(__file__)
"""

from pathlib import Path
#get the current path of our code
code_dir = Path().resolve()
#create output_dir within the same path
output_dir = os.path.join(code_dir, 'outputGANs/')
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

# Build GNN with defined vars and functions

In [10]:
# Put randomly generated sample Z into the generator to create "fake" images
G_sample, _ = generator(Z)
# The result of discriminator of real and fake samples
_, D_logit_real = discriminator(X)
_, D_logit_fake = discriminator(G_sample)

# generator loss 
# the goal of generator is to let discriminator make more mistakes on fake samples
# tf.ones_like returns a tensor with all elements set to 1
# 0 represent fake and 1 means real
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

# discriminator loss 
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)

In [11]:
def next_batch(batch_size = 5, from_test=False):
    
    if from_test:
        return images_test,labels_test
    else:
        index = np.random.choice(list(range(len(images_train))),size=batch_size)
        return images_train[index], labels_train[index]

# Start the session

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 128
# the dimension of the random samples
z_dim = 100
result_freq = 1000
# plot generators' output every figure_iter step
figure_iter = 1000
max_iter = 50000
size1 = 5
size2 = 5
i = 0

total_batch = int((train_num_examples+batch_size-1) / batch_size)

for iter in range(max_iter):
    
    if iter % figure_iter == 0:
        
        # G_sample is a sample from the generator
        samples = sess.run(G_sample, feed_dict={Z: sample_z(size1*size2, z_dim)})

        fig1 = plot_sample(samples, size1, size2)
        plt.savefig(output_dir + 'GANs' + str(i) + '.png', bbox_inches='tight')
        i += 1
        plt.close(fig1)

    j = iter%total_batch
    start = j * batch_size
    end = min(train_num_examples, start + batch_size)
    batch_xs = x_train[start:end]
    batch_xs = batch_xs.reshape(-1,3072)

    _, discriminator_loss = sess.run([D_solver, D_loss], feed_dict={X: batch_xs, Z: sample_z(batch_size, z_dim)})
    _, generator_loss     = sess.run([G_solver, G_loss], feed_dict={Z: sample_z(batch_size, z_dim)})

    if iter % result_freq == 0:
        print('iteration: {}'.format(iter))
        print('D_loss: {:0.4}'.format(discriminator_loss))
        print('G_loss: {:0.4}'.format(generator_loss))
        print()

iteration: 0
D_loss: 1.406
G_loss: 5.815

iteration: 1000
D_loss: 0.4738
G_loss: 3.623

iteration: 2000
D_loss: 1.434
G_loss: 1.868

iteration: 3000
D_loss: 2.265
G_loss: 1.374

iteration: 4000
D_loss: 1.296
G_loss: 2.638

iteration: 5000
D_loss: 1.39
G_loss: 0.87

iteration: 6000
D_loss: 2.237
G_loss: 0.8842

iteration: 7000
D_loss: 1.39
G_loss: 1.786

iteration: 8000
D_loss: 1.723
G_loss: 0.6481

iteration: 9000
D_loss: 0.9782
G_loss: 1.495

iteration: 10000
D_loss: 0.451
G_loss: 3.18

iteration: 11000
D_loss: 1.433
G_loss: 1.042

iteration: 12000
D_loss: 1.029
G_loss: 1.076

iteration: 13000
D_loss: 1.214
G_loss: 1.093

iteration: 14000
D_loss: 1.394
G_loss: 0.7071

iteration: 15000
D_loss: 0.9233
G_loss: 1.352

iteration: 16000
D_loss: 1.356
G_loss: 0.7046

iteration: 17000
D_loss: 1.31
G_loss: 0.676

iteration: 18000
D_loss: 1.386
G_loss: 0.6953

iteration: 19000
D_loss: 1.44
G_loss: 0.7798

iteration: 20000
D_loss: 1.386
G_loss: 0.6892

iteration: 21000
D_loss: 1.389
G_loss: 0.69